# Multiparameter TTI Bayesian Optimization - Minimizing R and the money lost for variable values of the probability of going to school, the testing delay, and the app trace delay

In [1]:
%cd tti-explorer

/Users/louis/Desktop/BayesianTTI/tti-explorer/notebooks/tti-explorer


In [2]:
import GPy
import GPyOpt
from numpy.random import seed
import matplotlib
import numpy as np
import math
import pandas as pd
import os
from tqdm.notebook import trange
from tti_explorer import config, utils
from tti_explorer.case import simulate_case, CaseFactors
from tti_explorer.contacts import EmpiricalContactsSimulator
from tti_explorer.strategies import TTIFlowModel, RETURN_KEYS
from tti_explorer.strategies import TTIFlowModel

def load_csv(pth):
    return np.loadtxt(pth, dtype=int, skiprows=1, delimiter=",")

path_to_bbc_data = os.path.join("data", "bbc-pandemic")
over18 = load_csv(os.path.join(path_to_bbc_data, "contact_distributions_o18.csv"))
under18 = load_csv(os.path.join(path_to_bbc_data, "contact_distributions_u18.csv"))

rng = np.random.RandomState(0)

# Hyperparameters

In [3]:
"""

defining global constants

"""

# other
N_CASES = 10000
LOST_DAILY_WAGE = 115
QUARANTINE_LENGTH = 10
TOTAL_TTI_COST = 22_000_000_000/365*10
UK_POPULATION = 69_000_000
TTI_COST = TOTAL_TTI_COST/UK_POPULATION * N_CASES
DAILY_MONEY_LOST_NO_SCHOOL = 1000

# probabilities
P_UNDER18 = 0.2
P_SYMPTOMATIC = 0.002
P_WORK_FROM_HOME = 0.3
P_INFECTED = 0.001
P_COMPLIANCE = 0.8

# bounds
MAX_R = 4
MAX_CONTACTS = 20
MAX_CASES = N_CASES*MAX_CONTACTS
MAX_TESTING_DELAY = 5
MAX_QUARANTINE_LENGTH = 14
MAX_MONEY = LOST_DAILY_WAGE * MAX_QUARANTINE_LENGTH * N_CASES + TTI_COST*1.3 + P_UNDER18*N_CASES*1000
MAX_COMPLIANCE = 0.8


MIN_CASES = 0
MIN_MONEY = 0
MIN_R = 0

# Defining a Simulation Function

The first step is to define the "simulation" function, which is going to run the covid-19 simulation given some parameter vector $\theta$. It will return a dictionary containing some useful information which is going to help us determine how successful the simulation was, i.e. it will directly affect the cost which we will assign later on when doing the Bayesian Optimization over $\theta$.  

In [4]:
def simulation(theta, config_details):
    
    """
    
        theta: list in R^d where d is the number of parameters
        config_details: dictionary where config_details[i] contains the associated
                        to the config file in configs and the name associated to theta (for changing the config)
                        
    """
    
    for i, val in enumerate(theta):
        config_type = config_details[i]["config"]
        config_name = config_details[i]["name"]
        configs[config_type][config_name] = val

    
    factor_config = utils.get_sub_dictionary(configs["policy_config"], config.DELVE_CASE_FACTOR_KEYS)
    strategy_config = utils.get_sub_dictionary(configs["policy_config"], config.DELVE_STRATEGY_FACTOR_KEYS)
    
    rng = np.random.RandomState(42)
    simulate_contacts = EmpiricalContactsSimulator(over18, under18, rng)
    tti_model = TTIFlowModel(rng, **strategy_config)
    
    outputs = list()

    for _ in trange(N_CASES):
        case = simulate_case(rng, **configs["case_config"])
        case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
        contacts = simulate_contacts(case, **configs["contacts_config"])
        res = tti_model(case, contacts, case_factors)
        outputs.append(res)
        
    to_show = [
        RETURN_KEYS.base_r,
        RETURN_KEYS.reduced_r,
        RETURN_KEYS.man_trace,
        RETURN_KEYS.app_trace,
        RETURN_KEYS.tests,
        RETURN_KEYS.cases_prevented_symptom_isolating,
        RETURN_KEYS.cases_prevented_social_distancing,
        RETURN_KEYS.cases_prevented_contact_tracing
    ]
    
    nppl = case_config['infection_proportions']['nppl']
    scales = [1, 1, nppl, nppl, nppl, nppl, nppl, nppl]

    return dict(pd.DataFrame(outputs).mean(0).loc[to_show].mul(scales))   

# Defining the Parameter Space to Optimize Over

For this optimization, we consider $\theta \in \mathbb{R}^3$. The three parameters that we are considering are:
- The probability of going to school
- The testing delay
- the app trace delay

In [28]:
types = {"go_to_school_prob" : "continuous",
         "testing_delay"     : "discrete",
         "quarantine_length"   : "discrete"}

bounds =[{'name': 'go_to_school_prob', 'type': types['go_to_school_prob'], 'domain': (0, 1)},
         {'name': 'testing_delay', 'type': types['testing_delay'], 'domain': (0, 5)},
         {'name': 'quarantine_length', 'type': types['quarantine_length'], 'domain': (1, 10)}]

config_details = {0:  {"name": "go_to_school_prob", "config": "policy_config"}, 
                  1:  {"name": "testing_delay", "config": "policy_config"},
                  2:  {"name": "quarantine_length", "config": "policy_config"}}

# Key Performance Indicators / Cost

To optimize over some parameter space, we first need to define some cost function. In this case, we will consider both $R$ as well as the money lost from people being forced into quarantine. In addition, we also define a function that gives us the number of cases prevented, another value that we could optimize (perhaps in a later cost function). 

In [29]:
def cases_prevented(x, output_dict):
    
    """
    
    returns the total number of cases prevented given the results of a simulation
    
    """
    
    cases_prevented = output_dict['# Secondary Infections Prevented by Contact Tracing'] \
                    + output_dict['# Secondary Infections Prevented by Social Distancing'] \
                    + output_dict['# Secondary Infections Prevented by Isolating Cases with Symptoms'] 

    return (cases_prevented-MIN_CASES)/(MAX_CASES-MIN_CASES)

In [30]:
def R(x, output_dict):
    
    """
    
    returns the effective R from the measures taken in the simulation
    
    """
    
    return (output_dict["Effective R"] - MIN_R)/(MAX_R-MIN_R)

In [31]:
def tti_cost(testing_delay):
    
    """
    
    returns the additional TTI cost due to the testing delay
    
    """
        
    if testing_delay  == 0: cost = TTI_COST*1.3
    if testing_delay  == 1: cost = TTI_COST*1.2
    if testing_delay  == 2: cost = TTI_COST*1.1
    if testing_delay  == 3: cost = TTI_COST*1.0
    if testing_delay  == 4: cost = TTI_COST*0.9
    if testing_delay  == 5: cost = TTI_COST*0.8
    
    return cost

In [32]:
def money_lost(x, output_dict):
    
    """
    
    returns the money lost given the results of a simulation
    
    """
    
    go_to_school_prob, testing_delay, quarantine_length = x
    
    # money lost because of compliant symptomatic people who cannot work from home are forced into quarantine
    # the total amount is also influenced by the testing and app delays since, supposing the person is actually 
    # negative, a quick test result will lead to less time in quarantine
    
    #lost_wages = (1 - P_UNDER18)*(1 - P_WORK_FROM_HOME)*LOST_DAILY_WAGE*P_COMPLIANCE*N_CASES*P_SYMPTOMATIC*(P_INFECTED*(app_trace_delay + QUARANTINE_LENGTH)+(1-P_INFECTED)*(app_trace_delay+testing_delay))    
    lost_wages = (1 - P_UNDER18)*(1 - P_WORK_FROM_HOME)*LOST_DAILY_WAGE*P_COMPLIANCE*N_CASES*P_SYMPTOMATIC*(P_INFECTED*quarantine_length + (1-P_INFECTED)*testing_delay)
    
    # money lost because kids are not going to school (1000 dollars per day per kid)
    missed_school_cost = P_UNDER18*N_CASES*DAILY_MONEY_LOST_NO_SCHOOL*(1 - go_to_school_prob)
    
    # cost of tti given the time taken to produce a test (will cost more if we need to produce tests more quickly)
    cost_of_tti = tti_cost(testing_delay)
    
    # add up all the costs
    lost_money = lost_wages + missed_school_cost + cost_of_tti
    
    return (lost_money-MIN_MONEY)/(MAX_MONEY-MIN_MONEY)

In [48]:
def total_cost_R(x):
    
    """
    
    returns the total cost, i.e. the money lost minus the number of cases prevented, 
    all scaled by lambda 
    
    
    """
    
    x = x[0]
    x_ = []
    for i in config_details.keys():
        if types[bounds[i]["name"]] == "discrete":
            x_.append(int(x[i]))
        else:
            x_.append(x[i])
            
    output_dict = simulation(x_, config_details)
    total_cost = lam*R(x_, output_dict) + (1-lam)*money_lost(x_, output_dict)
    
    print("theta:", x_)
    print("total cost:", total_cost)
    return total_cost

In [49]:
def total_cost_cases_prevented(x):
    
    """
    
    returns the total cost, i.e. the money lost minus the number of cases prevented, 
    all scaled by lambda 
    
    
    """
    
    x = x[0]
    x_ = []
    for i in config_details.keys():
        if types[bounds[i]["name"]] == "discrete":
            x_.append(int(x[i]))
        else:
            x_.append(x[i])
            
    output_dict = simulation(x_, config_details)
    total_cost = lam*cases_prevented(x_, output_dict) + (1-lam)*money_lost(x_, output_dict)
    
    print("theta:", x_)
    print("total cost:", total_cost)
    return total_cost

# Bayesian Optimization on the parameters

We are now ready to optimize over the parameter space to find the optimal value of $\theta$. 

In [51]:
def optimize(cost_function):
    
    """
    
    Runs Bayesian optimization on cost_function with the parameters defined elswehere
    
    """
    
    seed(42)
    print("Setting up optimization...")
    optimizer = GPyOpt.methods.BayesianOptimization(cost_function,
                                                  domain=bounds,
                                                  model_type = 'GP',
                                                  acquisition_type='EI',  
                                                  normalize_Y = True)    
    
    print("Starting optimization...")
    optimizer.run_optimization(max_iter = 10 , max_time = 100 , verbosity=True, eps=1e-06)
    
    # evaluate the result of the optimization
    optimizer.plot_acquisition()
    optimizer.plot_convergence()
    
    return evaluate(np.expand_dims(optimizer.x_opt, axis=0))

# Analyzing the Results

In [52]:
def evaluate(x):
    
    """
    
    displays some KPIs to evaluate the performance of some value determined through Bayesian
    Optimisation
    
    """
    
    x_ = []
    for i in config_details.keys():
        if types[bounds[i]["name"]] == "discrete":
            x_.append(int(x[0][i]))
        else:
            x_.append(x[0][i])
        
    output_dict = simulation(x_, config_details)
    
    prevented_cases = cases_prevented(x_, output_dict)*(MAX_CASES-MIN_CASES)+ MIN_CASES
    effective_R = R(x_, output_dict)
    lost_money = money_lost(x_, output_dict)*(MAX_MONEY-MIN_MONEY)+ MIN_MONEY
    cost_total = total_cost(x)
    
    print()
    print("####################################################")
    print("Key Performance Indicators")
    print("####################################################")
    print("Optimal value:", x_)
    print("Effective R:", effective_R)
    print("Cases Prevented: ", prevented_cases)
    print("Money Lost: ", lost_money)
    print("Total Cost: ", cost_total)

    # compute cost per prevented hospitalization
    p_hospitalisation = 0.1
    prevented_hospitalisations = p_hospitalisation * prevented_cases
    cost_per_prevented_hospitalisation = lost_money/prevented_hospitalisations

    print("Cost per prevented hospitalization: ", cost_per_prevented_hospitalisation)

In [53]:
def compare(values = [np.array([[1, 1, 10, 1]]), np.array([[1, 1, 1, 1]])]):
    
    """
    
    compares the KPIs for some range of values. Useful for debugging or fine-tuning the cost 
    functions prior to running some Bayesian Optimizations
    
    """
    
    for x in values:
        evaluate(x)

# Main (run different types of simulations)

## Cost Function takes into account R and the money lost (varying $\lambda$)

In [57]:
for name in ["S1_test_based_TTI", "S2_test_based_TTI", "S3_test_based_TTI", "S4_test_based_TTI", "S5_test_based_TTI"]:
    print("####################################################")
    # define config file for each simulation
    case_config = config.get_case_config("delve")
    contacts_config = config.get_contacts_config("delve")
    policy_config = config.get_strategy_configs("delve", name)[name]
    configs = {"case_config": case_config, "contacts_config": contacts_config, "policy_config": policy_config}
    
    print("Configuration:", name)
    print("Cost function:", "total_cost_R")
    print("Optimizing: ", [config_details[i]["name"] for i in range(3)])
    
    # run the optimization for different values of lambda
    for l in [0.1, 0.5, 0.9]:
        lam = l
        print("----------------------------------------------------------------------------") ; print()
        print("lambda = ", lam)
        optimize(cost_function=total_cost_R)
        print("####################################################") ; print()

####################################################
Configuration: S1_test_based_TTI
Cost function: total_cost_R
Optimizing:  ['go_to_school_prob', 'testing_delay', 'quarantine_length']
----------------------------------------------------------------------------

lambda =  0.1


A Jupyter Widget


theta: [0.15599452033620265, 0, 10]
total cost: 0.13182897167035468


A Jupyter Widget


theta: [0.05808361216819946, 5, 1]
total cost: 0.14524588938337848


...


theta: [0.7769078212473434, 5, 1]
total cost: 0.07851071752720849
num acquisition: 7, time elapsed: 55.32s


A Jupyter Widget


theta: [0.9994590671864021, 0, 10]
total cost: 0.0522099103347093
num acquisition: 8, time elapsed: 63.59s


A Jupyter Widget

KeyboardInterrupt: 

## Cost Function takes into account number of cases prevented and money lost

In [ ]:
for name in ["S1_test_based_TTI", "S2_test_based_TTI", "S3_test_based_TTI", "S4_test_based_TTI", "S5_test_based_TTI"]:

    # define config file for each simulation
    case_config = config.get_case_config("delve")
    contacts_config = config.get_contacts_config("delve")
    policy_config = config.get_strategy_configs("delve", name)[name]
    configs = {"case_config": case_config, "contacts_config": contacts_config, "policy_config": policy_config}
    
    print("Configuration:", name)
    print("Cost function:", "total_cost_cases_prevented")
    print("Optimizing: ", [config_details[i]["name"] for i in range(3)])
    
    # run the optimization for different values of lambda
    for l in [0.1, 0.5, 0.9]:
        lam = l
        print("lambda = ", lam)
        optimize(cost_function=total_cost_cases_prevented)
        print("####################################################")
        print()